In [ ]:
import re
from requests import get
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta, date
from time import sleep
import random
from openpyxl import Workbook
1
# 사용자 입력
keyword = '수소'
start_date = '1990.01.01'
end_date = '2024.12.31'

# 엑셀 파일 생성
wb = Workbook()
ws = wb.active
ws.title = keyword

# 엑셀 열 너비 조절
ws.column_dimensions['A'].width = 20
ws.column_dimensions['B'].width = 20
ws.column_dimensions['C'].width = 60
ws.column_dimensions['D'].width = 120
ws.column_dimensions['E'].width = 50
ws.column_dimensions['F'].width = 20  # 언론사명

# 엑셀 파일에 헤더 설정
ws.append(['날짜', '키워드', '네이버 뉴스제목', '네이버뉴스 내용', '네이버뉴스 링크', '언론사명'])

# 헤더 설정 - 웹사이트에 접속할 때 사용할 사용자 정보
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Referer': 'https://www.naver.com/',
}

# 날짜 범위를 3개월씩 나누는 함수
def get_date_ranges(start_date, end_date, delta=timedelta(days=90)):
    start_dt = datetime.strptime(start_date, '%Y.%m.%d')
    end_dt = datetime.strptime(end_date, '%Y.%m.%d')

    ranges = []
    while start_dt < end_dt:
        temp_end_dt = min(start_dt + delta, end_dt)
        ranges.append((start_dt.strftime('%Y.%m.%d'), temp_end_dt.strftime('%Y.%m.%d')))
        start_dt = temp_end_dt + timedelta(days=1)
    return ranges

# 문자열에서 불법 문자를 제거하는 함수 (정규 표현식으로 불법 문자 필터링)
def clean_string(value):
    # Excel에서 허용되지 않는 문자 및 잘못된 유니코드 문자 필터링
    ILLEGAL_CHARACTERS_RE = re.compile(r'[\x00-\x1F\x7F-\x9F\uD800-\uDFFF]')
    if isinstance(value, str):
        return ILLEGAL_CHARACTERS_RE.sub('', value)
    return value

# 뉴스 크롤링 함수 (최대 200페이지 크롤링)
def fetch_news_by_date_range(keyword, start_date, end_date):
    naver_news_links = []

    # 최대 200페이지(2000개 기사) 크롤링
    for page in range(1, 3001, 10):  # 1페이지부터 200페이지까지 (페이지당 10개 기사)
        print(f"페이지 {page // 10 + 1} 크롤링 중입니다.")
        url = f"https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{start_date.replace('.', '')}to{end_date.replace('.', '')}&start={page}"

        try:
            response = get(url, headers=headers)
            if response.status_code == 200:
                soup = bs(response.text, 'html.parser')
                news_items = soup.select('a.info')
                if not news_items:
                    break
                for item in news_items:
                    link = item['href']
                    if 'news.naver.com' in link:
                        naver_news_links.append(link)
                sleep(random.uniform(3, 5))  # 크롤링 간격을 3~5초 사이로 설정
            elif response.status_code == 403:
                print("403 에러 발생, 요청을 건너뜁니다.")
                sleep(random.uniform(60, 90))  # 짧게 대기 후 다음 요청
                continue
            else:
                print(f"Error fetching page: {response.status_code}")
                sleep(random.uniform(60, 90))
                continue
        except Exception as e:
            print(f"예상치 못한 에러 발생: {e}")
            sleep(random.uniform(60, 90))
            continue

    # 뉴스 본문 크롤링
    print("뉴스 본문 텍스트 크롤링을 시작합니다!")
    for idx, link in enumerate(naver_news_links):
        try:
            response = get(link, headers=headers)
            if response.status_code == 200:
                soup = bs(response.text, 'html.parser')
                title = soup.select_one('div.media_end_head_title').text.strip().replace('\n', '')
                content = soup.select_one('article#dic_area').text.strip().replace('\n', '')
                article_date = soup.select_one('span.media_end_head_info_datestamp_time').get_text(strip=True)
                press = soup.select_one('div.media_end_head_top a').text.strip()

                # 언론사명 중복 제거 (예: "경향일보경향일보" -> "경향일보")
                press = re.sub(r'\b(\w+)\b\s*\1', r'\1', press)

                # 데이터를 클린한 후 엑셀 파일에 추가
                ws.append([
                    clean_string(article_date),
                    clean_string(keyword),
                    clean_string(title),
                    clean_string(content),
                    clean_string(link),
                    clean_string(press)
                ])

                print(f"뉴스 제목: {title} {idx + 1}/{len(naver_news_links)}")
                sleep(random.uniform(2, 4))  # 본문 크롤링 간격 설정
            else:
                print(f"Error fetching article: {response.status_code}")
                sleep(random.uniform(2, 4))
        except Exception as e:
            print(f"Error fetching article at {link}: {e}")
            sleep(random.uniform(2, 4))

# 날짜 범위를 나누어 크롤링 실행
date_ranges = get_date_ranges(start_date, end_date)
for start, end in date_ranges:
    fetch_news_by_date_range(keyword, start, end)

# 엑셀 파일 저장
today_str = date.today().strftime("%Y%m%d")
wb.save(f'naver_news_{keyword}_{start_date}_{end_date}_{today_str}.xlsx')
print("Crawling and data extraction complete!")


페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
뉴스 본문 텍스트 크롤링을 시작합니다!
페이지 1 크롤링 중입니다.
